In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import pickle
import time

# Предполагаем, что у нас есть один из ваших датасетов, например, dataset_1.csv
df = pd.read_csv('dataset_1.csv')

# Подготовка данных
# Кодируем категориальные признаки
le_figure1 = LabelEncoder()
le_figure2 = LabelEncoder()
df['figure1'] = le_figure1.fit_transform(df['figure1'])
df['figure2'] = le_figure2.fit_transform(df['figure2'])

# Если есть дополнительные категориальные признаки (color1, color2)
if 'color1' in df.columns:
    le_color1 = LabelEncoder()
    le_color2 = LabelEncoder()
    df['color1'] = le_color1.fit_transform(df['color1'])
    df['color2'] = le_color2.fit_transform(df['color2'])

# Разделяем признаки и целевую переменную
X = df.drop('collision', axis=1)
y = df['collision']

# Разделяем на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Выбор 4 классических методов машинного обучения
models = {
    # Логистическая регрессия: 
    # - Простой и интерпретируемый метод
    # - Хорошо работает с бинарной классификацией
    # - Эффективен для линейно разделимых данных
    'Logistic Regression': LogisticRegression(max_iter=1000),
    
    # Дерево решений:
    # - Интуитивно понятен и легко интерпретируется
    # - Может моделировать нелинейные зависимости
    # - Хорошо подходит для смешанных типов данных
    'Decision Tree': DecisionTreeClassifier(max_depth=5),
    
    # k-ближайших соседей:
    # - Простой и не требует предположений о распределении данных
    # - Эффективен для задач с локальной структурой данных
    # - Подходит для небольших и средних датасетов
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    
    # Наивный Байес:
    # - Очень быстрый и требует мало данных для обучения
    # - Хорошо работает с независимыми признаками
    # - Прост в реализации и эффективен для базовой классификации
    'Naive Bayes': GaussianNB()
}

# Обучение и оценка моделей
results = {}
for name, model in models.items():
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    accuracy = model.score(X_test, y_test)
    results[name] = {'accuracy': accuracy, 'time': training_time}
    
    # Сохранение модели
    with open(f'{name.lower().replace(" ", "_")}_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    
    print(f"{name}: потребление = {accuracy:.4f}, Время обучения = {training_time:.4f} сек")

# Выбор 3 наиболее быстрых алгоритмов с низким потреблением данных
# Основываясь на характеристиках:
# 1. Naive Bayes - минимальное время обучения, низкие требования к данным
# 2. Logistic Regression - быстрая сходимость, работает с небольшими выборками
# 3. Decision Tree - быстрое обучение на небольших данных, не требует сложной предобработки

fastest_models = [
    'Naive Bayes',
    'Logistic Regression',
    'Decision Tree'
]

print("\nТри наиболее быстрых алгоритма с низким потреблением данных:")
for model_name in fastest_models:
    print(f"- {model_name}: потребление = {results[model_name]['accuracy']:.4f}, "
          f"время = {results[model_name]['time']:.4f} сек")

# Пример загрузки сохраненной модели
# with open('naive_bayes_model.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)

Logistic Regression: потребление = 0.6000, Время обучения = 0.0094 сек
Decision Tree: потребление = 0.7000, Время обучения = 0.0011 сек
K-Nearest Neighbors: потребление = 0.4000, Время обучения = 0.0012 сек
Naive Bayes: потребление = 0.8000, Время обучения = 0.0010 сек

Три наиболее быстрых алгоритма с низким потреблением данных:
- Naive Bayes: потребление = 0.8000, время = 0.0010 сек
- Logistic Regression: потребление = 0.6000, время = 0.0094 сек
- Decision Tree: потребление = 0.7000, время = 0.0011 сек
